In [1]:
import sys
sys.path.append('../src')
from db import *
from sqlalchemy import func
from sqlalchemy.orm import aliased
import numpy as np
from sqlalchemy import or_, and_
from tqdm import tqdm

unable to import 'smart_open.gcs', disabling that module
/Users/abuonomo/code/DataSquad/astro2020/venv/lib/python3.7/site-packages/tqdm/std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
db_loc = '../scratch/test.sqlite'

In [3]:
engine = create_engine(f"sqlite:///{db_loc}")
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
# no_below = 10
# no_above = 0.5
# min_mean_score = 0.01

# corpus_size = session.query(Paper).count()
# no_above_abs = int(no_above * corpus_size)
# kwd_query = (
#     session.query(
#         Keyword, func.count(Keyword.id), func.avg(PaperKeywords.score)
#     )
#     .join(PaperKeywords)
#     .join(Paper)
#     .group_by(Keyword.id)
# #     .order_by(func.count(Keyword.id).desc())
# #     .order_by(func.avg(PaperKeywords.score).desc())
#     .having(func.count() >= no_below)
#     .having(func.count() <= no_above_abs)
#     .having(func.avg(PaperKeywords.score) >= min_mean_score)
# )
# LOG.info(kwd_query)

In [5]:
# kwd_query.order_by(func.count(Keyword.id).desc())[0:10]

In [6]:
# kwd_query[0:10]

In [7]:
po = PaperOrganizer(no_below=10, no_above=0.5, min_mean_score=0.01)

In [41]:
year_min = min([y for y, c in po.get_year_counts(session)])

In [42]:
year_max = max([y for y, c in po.get_year_counts(session)])

In [34]:
po.get_year_counts(session)

[(1996, 394),
 (1997, 407),
 (1998, 423),
 (1999, 440),
 (2000, 427),
 (2001, 439),
 (2002, 434),
 (2003, 488),
 (2004, 490),
 (2005, 528),
 (2006, 559),
 (2007, 538),
 (2008, 515),
 (2009, 577),
 (2010, 586)]

In [22]:
for k, c, s in po.get_filtered_keywords(session)[0:5]:
    ys = po.get_kwd_years(session, k.keyword)
    print(k, ys)

<Keyword(keyword="black hole x-ray binary")> [(1997, 1), (1998, 1), (1999, 2), (2001, 1), (2005, 2), (2007, 1), (2009, 1), (2010, 1)]
<Keyword(keyword="dark energy model")> [(2002, 2), (2003, 1), (2004, 1), (2005, 4), (2006, 2), (2007, 4), (2008, 2), (2009, 1), (2010, 1)]
<Keyword(keyword="quasar luminosity function")> [(1997, 1), (2001, 2), (2003, 1), (2004, 1), (2006, 2), (2007, 1), (2008, 2), (2009, 1), (2010, 2)]
<Keyword(keyword="black hole binary")> [(1997, 2), (1999, 1), (2000, 1), (2006, 2), (2007, 1), (2009, 1), (2010, 4)]
<Keyword(keyword="low surface brightness galaxy")> [(1996, 1), (1997, 5), (1998, 1), (2001, 4), (2002, 2), (2003, 2), (2004, 1), (2005, 2), (2007, 1), (2008, 2), (2009, 1)]


In [37]:
ys = po.get_kwd_years(session, k.keyword)

In [56]:
yd = dict(ys)

In [57]:
fy = {}
for y in range(year_min, year_max):
    if y in yd:
        c = yd[y]
    else:
        c = 0
    fy[y] = c

In [58]:
fy

{1996: 1,
 1997: 5,
 1998: 1,
 1999: 0,
 2000: 0,
 2001: 4,
 2002: 2,
 2003: 2,
 2004: 1,
 2005: 2,
 2006: 0,
 2007: 1,
 2008: 2,
 2009: 1}

In [53]:
dict(ys)

{1996: 1,
 1997: 5,
 1998: 1,
 2001: 4,
 2002: 2,
 2003: 2,
 2004: 1,
 2005: 2,
 2007: 1,
 2008: 2,
 2009: 1}

In [50]:
list(zip(*ys))[0]

(1996, 1997, 1998, 2001, 2002, 2003, 2004, 2005, 2007, 2008, 2009)

In [51]:
1996 in list(zip(*ys))[0]

True

In [18]:
ys

[(1996, 1),
 (1997, 5),
 (1998, 1),
 (2001, 4),
 (2002, 2),
 (2003, 2),
 (2004, 1),
 (2005, 2),
 (2007, 1),
 (2008, 2),
 (2009, 1)]

In [11]:
po.get_kwd_years(session, "black hole")

[(1996, 5),
 (1997, 5),
 (1998, 4),
 (1999, 11),
 (2000, 5),
 (2001, 4),
 (2002, 3),
 (2003, 9),
 (2004, 9),
 (2005, 7),
 (2006, 10),
 (2007, 7),
 (2008, 7),
 (2009, 9),
 (2010, 11)]

In [7]:
nlp = get_spacy_nlp()

pm = PaperMiner(nlp)

pm.get_filtered_keywords(session)

INFO:db:SELECT keywords.id AS keywords_id, keywords.keyword AS keywords_keyword, count(keywords.id) AS count_1, avg(paper_keywords.score) AS avg_1 
FROM keywords JOIN paper_keywords ON keywords.id = paper_keywords.keyword_id JOIN papers ON papers.bibcode = paper_keywords.paper_bibcode GROUP BY keywords.id 
HAVING count(*) >= ? AND count(*) <= ? AND avg(paper_keywords.score) >= ? ORDER BY avg(paper_keywords.score) DESC


[(<Keyword(keyword="solar neutrino flux")>, 5, 0.08805533132992653),
 (<Keyword(keyword="black hole solution")>, 6, 0.06777864170716498),
 (<Keyword(keyword="dark energy perturbation")>, 5, 0.06457131502763466),
 (<Keyword(keyword="black hole entropy")>, 8, 0.06441613113808471),
 (<Keyword(keyword="high energy cosmic ray")>, 5, 0.06031572588700181),
 (<Keyword(keyword="galaxy stellar mass function")>, 8, 0.05916039950265407),
 (<Keyword(keyword="loop quantum gravity")>, 5, 0.05704066487378082),
 (<Keyword(keyword="massive star cluster")>, 6, 0.055648326003086),
 (<Keyword(keyword="Wilkinson Microwave Anisotropy Probe datum")>,
  5,
  0.05562826391134331),
 (<Keyword(keyword="black hole mass function")>, 5, 0.05559822606583661),
 (<Keyword(keyword="neutron star matter")>, 8, 0.05516433662967057),
 (<Keyword(keyword="standard solar model")>, 6, 0.05338508508229397),
 (<Keyword(keyword="black hole x-ray binary")>, 10, 0.051980075023631646),
 (<Keyword(keyword="dark matter density profile"

In [25]:
# session.query(PaperKeywords).join(Paper).filter(PaperKeywords.raw_keyword == "gamma").all()

In [26]:
year_counts = session.query(Paper.year, func.count(Paper.year)).group_by(Paper.year).all()

session.query(Keyword).filter(Keyword.keyword == "black hole").first().get_years(session)

session.query(Paper.year, func.count(Paper.year)) \
    .join(PaperKeywords) \
    .join(Keyword) \
    .filter(Keyword.keyword.ilike('mars')) \
    .group_by(Paper.year) \
    .all()

session.query(Keyword).filter(Keyword.keyword.ilike('universe'))[0].papers[1].paper.get_feature_text()

!pip install snakeviz

%load_ext snakeviz

paper_kwd = session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword=="measurement").first()

%%timeit -n 1 -r 1
added = 0
q = (
    session.query(Paper)
    .filter(~Paper.keywords.any(PaperKeywords.keyword_id == paper_kwd.keyword_id))
    .filter(
        or_(
            Paper.title.contains(paper_kwd.raw_keyword), 
            Paper.abstract.contains(paper_kwd.raw_keyword)
        )
    ).update
)
# %%timeit
# papers = q.all()
# with session.no_autoflush:
#     for p in papers:
#     #     with session.no_autoflush:
#         assoc = PaperKeywords(raw_keyword=paper_kwd.raw_keyword, score=np.nan)
#         assoc.keyword = paper_kwd.keyword
#         p.keywords.append(assoc)
#         added += 1
# session.rollback()

In [201]:
session.query(Keyword).count()

133320

In [237]:
session.rollback()

In [242]:
def get_pk_dict(p, paper_kwd):
    d = {
    'raw_keyword': paper_kwd.raw_keyword,
    'keyword_id': paper_kwd.keyword.id,
    'paper_bibcode': p.bibcode,
    }
    return d

In [11]:
session.query(PaperKeywords).group_by(PaperKeywords.raw_keyword).first()

<PaperKeywords(paper_bibcode="1997ApJ...486..665P", keyword.keyword="+")>

In [1]:
# %%timeit -n 1 -r 1
added = 0
t = 100
all_records = []
with session.no_autoflush:
    pbar = tqdm(session.query(PaperKeywords)[100:100+t], total=t)
    for paper_kwd in pbar:
        pbar.set_description(paper_kwd.raw_keyword)
        q = (
            session.query(Paper)
            .filter(~Paper.keywords.any(PaperKeywords.keyword_id == paper_kwd.keyword_id))
            .filter(
                or_(
                    Paper.title.contains(paper_kwd.raw_keyword), 
                    Paper.abstract.contains(paper_kwd.raw_keyword)
                )
            )
        )
        papers = q.all()
        for p in papers:
            d = get_pk_dict(p, paper_kwd)
            all_records.append(d)
            added += 1

NameError: name 'session' is not defined

In [255]:
len(all_records)

11830

In [256]:
%%timeit -n 1 -r 1
engine.execute(
    PaperKeywords.__table__.insert(),
    all_records,
)

280 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [18]:
session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword == 'measurement').count()

1133

In [13]:
print(session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword == 'measurement'))

SELECT paper_keywords.paper_bibcode AS paper_keywords_paper_bibcode, paper_keywords.keyword_id AS paper_keywords_keyword_id, paper_keywords.score AS paper_keywords_score, paper_keywords.raw_keyword AS paper_keywords_raw_keyword 
FROM paper_keywords JOIN keywords ON keywords.id = paper_keywords.keyword_id 
WHERE keywords.keyword = ?


In [222]:
engine

Engine(sqlite:///../scratch/test.sqlite)

In [216]:
session.flush()

In [150]:
session.query(PaperKeywords).join(Keyword).filter(Keyword.keyword == 'measurement').count()

307

In [148]:
added

15513

In [14]:
# session.rollback()
session.close()

In [71]:
q.first()

('2010MNRAS.409..531R')

In [52]:
KeywordAlias = aliased(Keyword)
q = (
 session.query(Keyword, func.count(Keyword.id))
    .join(PaperKeywords)
    .join(Paper)
    .group_by(Keyword.id)
#     .filter(Paper.year == 2010)
    .order_by(func.count(Keyword.id).desc())
    .having(func.count() > 10)
    .having(funct.count() < )
)

In [50]:
print(q)

SELECT keywords.id AS keywords_id, keywords.keyword AS keywords_keyword, count(keywords.id) AS count_1 
FROM keywords JOIN paper_keywords ON keywords.id = paper_keywords.keyword_id JOIN papers ON papers.bibcode = paper_keywords.paper_bibcode GROUP BY keywords.id 
HAVING count(*) > ? ORDER BY count(keywords.id) DESC


In [51]:
q.count()

3225

In [11]:
session.query(Paper).count()

7245

In [37]:
# q[0:100]

In [12]:
tokens = ([pk.keyword.keyword for pk in paper.keywords] for paper in session.query(Paper).all())

In [14]:
nlp = get_spacy_nlp()

In [22]:
pm = PaperMiner(nlp)

In [23]:
pm.get_dictionary(session)

In [32]:
corpus = pm.get_corpus(session)

In [70]:
session.rollback()

In [71]:
session.close()